In [1]:
# install diffusers from source if needed
# as the fluxpipeline is brand new

# !pip install git+https://github.com/huggingface/diffusers

In [2]:
# !pip uninstall transformer-engine -y

In [3]:
# !pip install transformers

In [4]:
# !pip install sentencepiece

In [5]:
# from huggingface_hub import notebook_login
# notebook_login()

In [6]:
# !pip uninstall apex -y

## doesnt work (37-48mins estimated for a single image)

In [7]:
# import torch
# from diffusers import FluxPipeline

# pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
# pipe.enable_model_cpu_offload()

# prompt = "a tiny astronaut hatching from an egg on the moon"
# out = pipe(
#     prompt=prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     num_inference_steps=25,
# ).images[0]
# out.save("image.png")

## FP8

In [8]:
# !pip install optimum-quanto

In [9]:
# !pip install -U torchvision

In [10]:
# !pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu121

### setup pipeline

In [11]:
import torch
from diffusers import FluxTransformer2DModel, FluxPipeline
from transformers import T5EncoderModel, CLIPTextModel
from optimum.quanto import freeze, qfloat8, quantize

bfl_repo = "black-forest-labs/FLUX.1-dev"
dtype = torch.bfloat16

transformer = FluxTransformer2DModel.from_single_file("https://huggingface.co/Kijai/flux-fp8/blob/main/flux1-dev-fp8.safetensors", torch_dtype=dtype)
quantize(transformer, weights=qfloat8)
freeze(transformer)

text_encoder_2 = T5EncoderModel.from_pretrained(bfl_repo, subfolder="text_encoder_2", torch_dtype=dtype)
quantize(text_encoder_2, weights=qfloat8)
freeze(text_encoder_2)

pipe = FluxPipeline.from_pretrained(bfl_repo, transformer=None, text_encoder_2=None, torch_dtype=dtype)
pipe.transformer = transformer
pipe.text_encoder_2 = text_encoder_2

pipe.enable_model_cpu_offload()

prompt = "A cat holding a sign that says hello world"
image = pipe(
    prompt,
    guidance_scale=1.5,
    height=1024,
    width=1024,
    output_type="pil",
    num_inference_steps=20,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]

image.save("flux-fp8-dev.png")

/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


  0%|          | 0/20 [00:00<?, ?it/s]

### test pipeline

In [12]:
# prompt = "A cat holding a sign that says 'i am stupid'"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=20,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("flux-fp8-dev-2.png")

In [13]:
# prompt = "A cat holding a sign that says 'i love cheese'"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=20,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("flux-fp8-dev-3.png")

In [14]:
# prompt = "picture of a delicious cheeseburger with fries on the side, in the forest, nature background"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=20,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("flux-fp8-dev-4.png")

In [15]:
# prompt = "Amuse-Bouche Crispy Parmesan waffle with tomato and basil gelée, 4k food photography, top down picture of a plate, michelin starred dish"
# image = pipe(
#     prompt,
#     guidance_scale=1.5,
#     height=1024,
#     width=1024,
#     output_type="pil",
#     num_inference_steps=25,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("food-test-2.png")

In [17]:
prompt = "Amuse-Bouche Crispy Parmesan waffle with tomato and basil gelée, 4k food photography, top down picture of a plate, michelin starred dish"
image = pipe(
    prompt,
    guidance_scale=1.5,
    height=1024,
    width=1024,
    output_type="pil",
    num_inference_steps=25,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]

image.save("food-test-4.png")

  0%|          | 0/25 [00:00<?, ?it/s]

We find that 25 steps with the above prompt template is the ideal config to run all 288 food prompts.  

### Process all 288 prompts

We load a datasets of prompts and filenames and pass it to the image generation function

In [19]:
# we set the seed to a fixed 42
def from_prompt_to_file(prompt, filename):
    image = pipe(
        prompt,
        guidance_scale=1.5,
        height=1024,
        width=1024,
        output_type="pil",
        num_inference_steps=25,
        generator=torch.Generator("cpu").manual_seed(42)
    ).images[0]

    image.save('images/'+filename)

In [27]:
# load the prompt dataset
import pandas as pd

prompt_df = pd.read_csv('food_prompts_and_filenames.csv', index_col=0)
prompt_df.head(5)

,prompt,filename
0,Amuse-Bouche Crispy Parmesan waffle with tomat...,Chef_Laurent_Dupont-L'Etoile_d'Or-Amuse-Bouche...
1,Appetizer Caramelized scallops with cauliflowe...,Chef_Laurent_Dupont-L'Etoile_d'Or-Appetizer-Ca...
2,Soup Velvety butternut squash bisque with crèm...,Chef_Laurent_Dupont-L'Etoile_d'Or-Soup-Velvety...
3,Fish Pan-seared turbot with Jerusalem artichok...,Chef_Laurent_Dupont-L'Etoile_d'Or-Fish-Pan-sea...
4,"Meat Roasted duck breast with figs, purple pot...",Chef_Laurent_Dupont-L'Etoile_d'Or-Meat-Roasted...


In [ ]:
from tqdm.auto import tqdm

tqdm.pandas()

prompt_df.progress_apply(lambda row: from_prompt_to_file(row['prompt'], row['filename']), axis=1)

  0%|          | 0/289 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]